### Linear SVC Model

In [1]:
import pandas as pd
from sklearn.svm import LinearSVC

In [2]:
tweet_df_location = pd.read_csv('../data/graphAnalysis/clean_climateTwitterData.csv')

In [3]:
len(tweet_df_location)

72405

In [4]:
tweet_df_location = tweet_df_location[tweet_df_location['text'].notna()]

In [5]:
len(tweet_df_location)

72405

In [6]:
tweet_df_location.groupby('search_hashtags')['id'].count()

search_hashtags
#actonclimate         1895
#bushfires             899
#climateaction        6378
#climatechange       16190
#climatecrisis        4982
#climatestrike       18355
#environment          4703
#fridaysforfuture     3038
#globalwarming        4152
#greennewdeal         4589
#savetheplanet        1434
#sustainability       5790
Name: id, dtype: int64

In [7]:
import re
import string
from nltk.tokenize import TweetTokenizer

# This step was performed in the sentiment Analyzer
# def replace_urls(in_string, replacement=None):
#     # """Replace URLs in strings. See also: ``bit.ly/PyURLre``

#     # Args:
#     #     in_string (str): string to filter
#     #     replacement (str or None): replacment text. defaults to '<-URL->'

#     # Returns:
#     #     str
#     # """
#     replacement = '<-URL->' if replacement is None else replacement
#     pattern = re.compile('(https?://)?(\w*[.]\w+)+([/?=&]+\w+)*')
#     return re.sub(pattern, replacement, in_string)


def tokenize_only(in_string):
    """
    Convert `in_string` of text to a list of tokens using NLTK's TweetTokenizer
    """
    # reasonable, but adjustable tokenizer settings
    tokenizer = TweetTokenizer(preserve_case=False,
                               reduce_len=True,
                               strip_handles=False)
    tokens = tokenizer.tokenize(in_string)
    return tokens

# #remove punctuations
# def rm_punctuation(tweet):
#     RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])  
#     print(f'this is the punctuation  {RE_PUNCTUATION}')
#     print("\n")
#     newValue = tweet.replace(RE_PUNCTUATION, "")
#     return newValue




In [8]:
# This step was performed in the sentiment Analyzer
# # remove urls and retweets and entities from the text
# tweet_df_location['text_clean'] = tweet_df_location['text'].apply(lambda row:replace_urls(row))

# #remove punctuations
# RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])  
# tweet_df_location['text_clean'] = tweet_df_location['text_clean'].str.replace(RE_PUNCTUATION, "")


In [9]:
from nltk.corpus import stopwords

# List of stopwords
stop_words= stopwords.words('english') #import stopwords from NLTK package
readInStopwords = pd.read_csv("pre_process/twitterStopWords.csv", encoding='ISO-8859-1') # import stopwords from CSV file as pandas data frame
readInStopwords = readInStopwords.wordList.tolist() # convert pandas data frame to a list

readInStopwords.append('http')
readInStopwords.append('https')

search_terms = ['#climateStrike','#climatestrike','#climatechange','#GreenNewDeal','#climatecrisis','#climateAction','#FridaysForFuture',
            '#environment','#globalwarming','#GlobalWarming','#ActOnClimate','#sustainability','#savetheplanet',
        '#bushfiresAustralia','#bushfires']

readInStopwords.extend(search_terms)
stop_list = stop_words + readInStopwords # combine two lists i.e. NLTK stop words and CSV stopwords
stop_list = list(set(stop_list)) # strore only unique values 
    
print(stop_list)

['am', 'isn', ' what', " would've", " when'll", ' of', '#climatecrisis', 'yours', ' in', 'above', ' however', ' so', ' some', ' did', 'were', 'then', "didn't", " ain't", 'those', 'y', ' this', 'only', " he'll", 'and', " they're", 'the', 'had', 'which', 'is', " we're", " they'd", ' any', ' your', 'to', " who's", 'can', 'her', 'more', ' its', ' these', ' a', 'be', 'such', " when'd", " she'll", 'hadn', ' be', 'll', " we'll", '#climatestrike', ' while', 'theirs', "i've", "you'd", 'both', 'most', ' at', 'himself', " couldn't", "should've", ' should', 'during', ' than', "mightn't", "we're", ' across', " what'd", ' been', " might've", ' said', " we'd", 'could', 'ain', ' other', "'tis", '#GreenNewDeal', ' from', " wouldn't", ' him', 'our', "when's", " mightn't", 'yourselves', " aren't", '#sustainability', " weren't", ' cannot', 'hers', " he'd", ' on', ' with', ' let', "you've", "there's", ' is', " he's", 'its', 'of', "you'll", 'weren', 'https', 'that', 's', 'over', ' neither', 'shouldn', " you

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X = tweet_df_location['text_clean']
y = tweet_df_location['search_hashtags']


In [12]:
# from sklearn.preprocessing import LabelEncoder / Not Required

# # Step 1: Label-encode data set
# label_encoder = LabelEncoder()
# label_encoder.fit(y)
# encoded_y = label_encoder.transform(y)
# encoded_y.shape
# encoded_y

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=0.00, stop_words=stop_list, tokenizer=tokenize_only) # Use tf (raw term count) features for LDA.
X_Vect = tf_vectorizer.fit_transform(X)


c:\users\tasne\envs\newproject\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['#actonclimate', '#bushfiresaustralia', '#climateaction', '#fridaysforfuture', '#greennewdeal', "'", 'able', 'across', "ain't", 'almost', 'also', 'among', "can't", 'cannot', "could've", 'dear', 'either', 'else', 'ever', 'every', 'get', 'got', "how'd", "how'll", 'however', 'least', 'let', 'like', 'likely', 'may', 'might', "might've", 'must', "must've", 'neither', 'often', 'rather', 'said', 'say', 'says', 'since', 'tis', 'twas', 'us', 'wants', "what'd", "when'd", "when'll", "where'd", "where'll", "who'd", "who'll", "why'd", "why'll", "would've", 'yet'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [14]:
X_Vect.shape

(72405, 87818)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_Vect, y, test_size=0.30, random_state=42, stratify=y)

In [16]:
X_train.shape

(50683, 87818)

In [17]:
y_train.shape

(50683,)

In [18]:
X_test.shape

(21722, 87818)

In [19]:
y_test.shape

(21722,)

In [20]:
svm = LinearSVC()
svm.fit(X_train, y_train)  
predictions = svm.predict(X_test)

In [21]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

[[ 420    1   17   40   26   30    5    6    1   15    0    7]
 [   0  226    2   29    6    1    2    1    3    0    0    0]
 [  15    3 1213  222   61  241   29   42   26   29    7   25]
 [  37   18  211 3828  141  247   84   29  131   59   22   50]
 [  16    9   78  156 1009  118   17   13   27   41    4    7]
 [  15    5   96  135   56 4986   16  107   27   41    9   14]
 [   9    0   13   64   10   14 1189    1   27    3   10   71]
 [   4    3   35   31   10  318    5  483    6    4    6    6]
 [   2    0    5   65   18   33   18    1 1089    5    5    5]
 [  14    0   13   55   27   40    4    7    6 1199    6    6]
 [   1    0    0   10    5    7   20    1   14    0  357   15]
 [   4    0   11   38    8    8   60    5   19    3   12 1569]]
                   precision    recall  f1-score   support

    #actonclimate       0.78      0.74      0.76       568
       #bushfires       0.85      0.84      0.84       270
   #climateaction       0.72      0.63      0.67      1913
   #cl

In [22]:
from sklearn import metrics
metrics.accuracy_score(y_test,predictions)

0.8087653070619648

- Accuracy is -- 0.8101924316361293 -- without tokens and stopwords
- Accuracy is -- 0.8087653070619648 -- with tokenization and stopwords
> Note adding or remove tokens and stopwords does not effect  accuracy much

In [23]:
strVal = ["Cuyahoga county eliminates use of plastic bags","Weather is still warm in winters and is not freezing"]
Value = tf_vectorizer.transform(strVal)
Value.shape

(2, 87818)

In [24]:
predictions = svm.predict(Value)

In [25]:
print(predictions)

['#environment' '#climatechange']


In [26]:
#Use this way to obtain the value from encoding
# inv_s = label_encoder.inverse_transform(predictions.astype(int).ravel())
# inv_s

In [27]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1]}
grid = GridSearchCV(LinearSVC(), param_grid)
grid.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)

Best cross-validation score: 0.84
Best parameters:  {'C': 0.01}


- Best cross-validation score: 0.84
- Best parameters:  {'C': 0.01}

In [28]:
grid.predict(Value)

array(['#environment', '#climatechange'], dtype=object)